In [2]:
import importlib
# --------------------
from sklearn.utils import gen_batches
from sklearn.model_selection import GridSearchCV, KFold, TimeSeriesSplit
import numpy as np
from numpy.linalg import norm,inv,matrix_rank
import h5py

# ----------
from sklearn.cross_decomposition import PLSRegression
from pytictoc import TicToc
tictoc=TicToc()

In [3]:
tmp = h5py.File('./data/TW_PM25.h5', 'r')
X_train, Y_train = tmp['X_train'], tmp['Y_train']
n_train = X_train.shape[0]

X_train = X_train[0:n_train]
Y_train = Y_train[0:n_train]
tmp.close()

In [3]:
print(X_train.shape)
print(Y_train.shape)

tscv = TimeSeriesSplit(n_splits=17, test_size=30)
for i, (train_index, test_index) in enumerate(tscv.split(X_train)):
    print(f"Fold {i}:")
    print(f"  Train: len={len(train_index)}")
    print(f"  Test:  len={len(test_index)}")
    if(i>3):
        break

(877, 1800)
(877, 1800)
Fold 0:
  Train: len=367
  Test:  len=30
Fold 1:
  Train: len=397
  Test:  len=30
Fold 2:
  Train: len=427
  Test:  len=30
Fold 3:
  Train: len=457
  Test:  len=30
Fold 4:
  Train: len=487
  Test:  len=30


In [4]:
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr, data
from rpy2.robjects.vectors import StrVector
from rpy2.robjects import numpy2ri
numpy2ri.activate()

bigsgpls = importr("bigsgPLS")
stats = importr("stats")
pls = bigsgpls.bigsgpls(X_train[:-100], Y_train[:-100], case=1, H=7, scale=False)


In [5]:
print(pls.rx2('variates').rx2('X').shape)
print(pls.rx2('scales').rx2('x_scale'))
print(pls.rclass)
print(pls.names)

print(pls.rx2('loadings').rx2('X').shape)
print(pls.rx2('scales').rx2('x_means').shape)
print(pls.rx2('scales').rx2('y_means').shape)

(777, 7)
[8.11062024 8.08642901 6.66353196 ... 9.25611338 5.5254836  4.56575435]
<rpy2.rinterface_lib.sexp.StrSexpVector object at 0x7f98ed20e500> [RTYPES.STRSXP]
['adjloadings' 'loadings' 'CEmat' 'variates' 'ncomp' 'scales' 'Y']
(1800, 7)
(1800,)
(1800,)


In [6]:
ypred = robjects.r.predict(pls, X_train[-100:], ng=1)

R[write to console]: Error in scale.default(B, center = FALSE, scale = 1/y_scale) : 
  length of 'scale' must equal the number of columns of 'x'



R[write to console]: In addition: 


R[write to console]: There were 33 warnings (use warnings() to see them)


R[write to console]: 



RRuntimeError: Error in scale.default(B, center = FALSE, scale = 1/y_scale) : 
  length of 'scale' must equal the number of columns of 'x'


In [ ]:
n_comp_lst=list(range(1,16))
parameters = {'n_components':n_comp_lst}
clf = GridSearchCV(PLSRegression(), parameters,
                   cv=tscv, n_jobs=4,
                   scoring='neg_root_mean_squared_error')
print(clf)
tictoc.tic()
clf.fit(X_train, Y_train)
print("CV elapsed time: {:.2f}s".format(tictoc.tocvalue()))
print("best parameter: ", clf.best_params_,
      "; score: {:15.7e}".format(clf.best_score_))